# Obtendo os dados

In [ ]:
 # Obtendo os dados e colocando-as em um dataframe
def obter_dados_bd():
    sql_select_dados = "select eua.id, eua.curriculo_texto, eva.id, eva.texto, na.nota \
    from nota_analise na inner join emprega_usuario_analise eua on na.usuario_id = eua.id \
    inner join emprega_vaga_analise eva on na.vaga_id = eva.id \
    order by na.vaga_id"
    curriculos = []
    vagas = []
    notas = []
    with nova_conexao() as conexao:
        try:
            cursor = conexao.cursor()
            cursor.execute(sql_select_dados)
            resultado = cursor.fetchall()
        except ProgrammingError as e:
            print(f'Erro: {e.msg}')
        else:
            for row in resultado:
                curriculos.append(row[1])
                vagas.append(row[3])
                notas.append(row[4])
            conexao.commit()

    df_dados = pd.DataFrame(
        {
            'curriculos' : curriculos,
            'vagas' : vagas,
            'notas' : notas
        })
    df_dados.to_csv('dados.csv', index=False, encoding='utf-8')
    return df_dados


# Lendo e salvando para CSV
def obter_dados_csv():
    df_dados = pd.read_csv('dados.csv')
    df_dados.to_csv('dados.csv', index=False, encoding='utf-8')
    return df_dados

In [ ]:
df_dados = obter_dados_csv()
df_dados

,curriculos,vagas,notas
0,Nome Completo: Ana Paula Santos Formação: B...,Desenvolvedor Full Stack Desenvolver soluções ...,1
1,"Rafael Santos Avenida das Estrelas, 789 - Cida...",Desenvolvedor Full Stack Desenvolver soluções ...,2
2,"Amanda Costa Rua das Flores, 123 - Cidade das ...",Desenvolvedor Full Stack Desenvolver soluções ...,5
3,"Leonardo Santos Avenida das Estrelas, 789 - Ci...",Desenvolvedor Full Stack Desenvolver soluções ...,4
4,"Gabriela Lima Rua dos Programadores, 123 - Cid...",Desenvolvedor Full Stack Desenvolver soluções ...,5
...,...,...,...
6195,"Nome: Ana Santos Endereço: Rua das Análises, 1...","Designer Gráfico Criar layouts, peças gráficas...",1
6196,Nome: Daniel Oliveira Endereço: Avenida dos Ec...,"Designer Gráfico Criar layouts, peças gráficas...",1
6197,Nome: Pedro Almeida Endereço: Rua das Finanças...,"Designer Gráfico Criar layouts, peças gráficas...",1
6198,Nome: Laura Costa Endereço: Avenida da Economi...,"Designer Gráfico Criar layouts, peças gráficas...",1


# Removendo os dígitos

In [ ]:
df_dados["curriculos"] = df_dados["curriculos"].apply(lambda x: re.sub('\d+', '', x))
df_dados["vagas"] = df_dados["vagas"].apply(lambda x: re.sub('\d+', '', x))
df_dados["notas"] = df_dados["notas"].apply(lambda x: x-1) # Ajustando o target porque o torch utiliza as classes começando do 0 [0, num_labels-1]
df_dados

,curriculos,vagas,notas
0,Nome Completo: Ana Paula Santos Formação: B...,Desenvolvedor Full Stack Desenvolver soluções ...,0
1,"Rafael Santos Avenida das Estrelas, - Cidade ...",Desenvolvedor Full Stack Desenvolver soluções ...,1
2,"Amanda Costa Rua das Flores, - Cidade das Art...",Desenvolvedor Full Stack Desenvolver soluções ...,4
3,"Leonardo Santos Avenida das Estrelas, - Cidad...",Desenvolvedor Full Stack Desenvolver soluções ...,3
4,"Gabriela Lima Rua dos Programadores, - Cidade...",Desenvolvedor Full Stack Desenvolver soluções ...,4
...,...,...,...
6195,"Nome: Ana Santos Endereço: Rua das Análises, ...","Designer Gráfico Criar layouts, peças gráficas...",0
6196,Nome: Daniel Oliveira Endereço: Avenida dos Ec...,"Designer Gráfico Criar layouts, peças gráficas...",0
6197,Nome: Pedro Almeida Endereço: Rua das Finanças...,"Designer Gráfico Criar layouts, peças gráficas...",0
6198,Nome: Laura Costa Endereço: Avenida da Economi...,"Designer Gráfico Criar layouts, peças gráficas...",0


# Configurando o Transformer e o Tokenizer

In [ ]:
# Transforma em um dataset do HuggingFace para usar a sua função map
dados = Dataset.from_pandas(df_dados)
dados

Dataset({
    features: ['curriculos', 'vagas', 'notas'],
    num_rows: 6200
})

In [ ]:
def tokenize_function(inputs):
    return tokenizer(
        inputs['curriculos'], inputs['vagas'], truncation=True, return_tensors='pt', padding='max_length', max_length=128
        )


tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
dados_tokenizados = dados.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=5, hidden_dropout_prob = 0.1, output_hidden_states=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Map:   0%|          | 0/6200 [00:00<?, ? examples/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
dados_tokenizados = dados_tokenizados.remove_columns(['curriculos', 'vagas'])
dados_tokenizados = dados_tokenizados.rename_column('notas', 'labels')
dados_tokenizados.set_format("torch")

In [ ]:
dados_tokenizados

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6200
})

# Preparando o treinamento

In [ ]:
def evaluate_model(eval_dataloader):

    metric_acc = evaluate.load("accuracy")
    metric_f1_micro = evaluate.load("f1")
    metric_f1_macro = evaluate.load("f1")
    metric_f1_weighted = evaluate.load("f1")
    metric_roc_auc = evaluate.load("roc_auc", "multiclass")
    progress_bar2 = tqdm(range(len(eval_dataloader)), desc='Validação')
    model.eval()

    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=1)

        metric_acc.add_batch(predictions=predictions, references=batch["labels"])
        metric_f1_micro.add_batch(predictions=predictions, references=batch["labels"])
        metric_f1_macro.add_batch(predictions=predictions, references=batch["labels"])
        metric_f1_weighted.add_batch(predictions=predictions, references=batch["labels"])
        metric_roc_auc.add_batch(predictions=predictions, references=batch["labels"])
        
        progress_bar2.update(1)
        
    acc = metric_acc.compute()
    f1_micro = metric_f1_micro.compute(average="micro")
    f1_macro = metric_f1_macro.compute(average="macro")
    f1_weighted = metric_f1_weighted.compute(average="weighted")
    roc_auc =  metric_roc_auc.compute()
    
    return acc, f1_micro, f1_macro, f1_weighted, roc_auc

In [ ]:
dados_tokenizados=dados_tokenizados.train_test_split(0.2, seed=42)
test_dataloader = DataLoader(dados_tokenizados['test'], batch_size=8)

# A partir do treino, obtém os dados de validação
dados_tokenizados=dados_tokenizados['train'].train_test_split(0.2, seed=42)
train_dataloader = DataLoader(dados_tokenizados['train'], shuffle=True, batch_size=8)
val_dataloader = DataLoader(dados_tokenizados['test'], batch_size=8)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
optimizer = AdamW(model.parameters(), lr=5e-5)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

dados_tokenizados

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3968
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 992
    })
})

# Treinamento

In [ ]:
progress_bar = tqdm(range(num_training_steps), desc='Treino')
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    train_acc, train_f1_micro, train_f1_macro, train_f1_weighted = evaluate_model(train_dataloader)
    val_acc, val_f1_micro, val_f1_macro, val_f1_weighted = evaluate_model(val_dataloader)
    print(f'Resultados com o conjunto de treino na época {epoch}')
    print(f'Acurácia: {train_acc}')
    print(f'Micro-F1: {train_f1_micro}')
    print(f'Macro-F1: {train_f1_macro}')
    print(f'Weighted-F1: {train_f1_weighted}')

    print(f'Resultados com o conjunto de validação na época {epoch}')
    print(f'Acurácia: {val_acc}')
    print(f'Micro-F1: {val_f1_micro}')
    print(f'Macro-F1: {val_f1_macro}')
    print(f'Weighted-F1: {val_f1_weighted}')

Treino:   0%|          | 0/1488 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
    # a melhor métrica será macro f1 porque o dataset está desbalanceado
    # para a métrica f1_weighted, devemos selecionar melhor os dados de validação para colocar mais amostras onde se tem nota 5